## Pass2Path Network - Train

In [1]:
# imports
import numpy as np
import tensorflow as tf
import os
import pickle
import copy
import string
import json
import csv
import time
import math
from word2keypress import Keyboard
from ast import literal_eval
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops
from pathlib import Path
from edit_distance_backtrace import find_med_backtrace
import random
from collections import OrderedDict
from pass2path_model import Pass2PathModel

In [2]:
'''
Preprocessing:
We need to create look-up tables in order to translate characters to mathematical representation.
'''

# CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }
CODES = {'<GO>': 0, '<EOS>': 1, '<UNK>': 2}
# chars = list(string.ascii_letters) + list(string.punctuation) +
# list(string.digits) + [" ", "\t", "\x03", "\x04"]
def create_lookup_tables_from_lst(char_lst):
    '''
    This function creates a dictionary out of a list with the added codes representing padding,
    unknows components, start and end of a sequence
    '''
    # <EOS> acts as <PAD>
    # CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }
    # chars = list(string.ascii_letters) + list(string.punctuation) +
    # list(string.digits) + [" ", "\t", "\x03", "\x04"]

    # make a list of unique chars (from https://stackoverflow.com/a/480227)
    seen = set()
    seen_add = seen.add
    vocab = [x for x in char_lst if not (x in seen or seen_add(x))]

    if len(vocab) < 100:
        print(vocab)
        print(char_lst)
    # (1)
    # starts with the special tokens
    vocab_to_int = copy.copy(CODES)

    # the index (v_i) will starts from 4 (the 2nd arg in enumerate() specifies
    # the starting index)
    # since vocab_to_int already contains special tokens
    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[str(v)] = v_i # opposite would be int()
    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab

def build_path_vocab(origin_vocab):
    '''
    This functions combines the path dictionary with the added codes.
    '''
    int_lst = sorted(list(origin_vocab.values()))
    path_to_int_vocab, int_to_path_vocab = create_lookup_tables_from_lst(int_lst)
    return path_to_int_vocab, int_to_path_vocab

In [19]:
# Globals
kb = Keyboard()
# thisfolder = Path(__file__).absolute().parent
# TRANS_to_IDX = json.load((thisfolder / 'data/trans_dict_2idx.json').open()) #
# py
with open(('./data/trans_dict_2idx.json')) as f:
    TRANS_to_IDX = json.load(f) # jupyter
IDX_to_TRANS = {v: literal_eval(k) for k, v in TRANS_to_IDX.items()}
char_lst = list(string.ascii_letters) + list(string.digits) + list(string.punctuation) + [" ", "\t", "\x03", "\x04"]
source_vocab_to_int, source_int_to_vocab = create_lookup_tables_from_lst(char_lst)
target_vocab_to_int, target_int_to_vocab = build_path_vocab(TRANS_to_IDX)
NUM_SAMPLES_LARGE_DS = 117574260
NUM_SAMPLES_CORNELL_DS = 11667608
tf.app.flags.FLAGS.__flags.clear()

# Network parameters
tf.app.flags.DEFINE_string('cell_type', 'lstm', 'RNN cell for encoder and decoder, default: lstm')
# tf.app.flags.DEFINE_string('attention_type', 'bahdanau', 'Attention mechanism: (bahdanau, luong), default: bahdanau')
tf.app.flags.DEFINE_integer('hidden_units', 128, 'Number of hidden units in each layer')
tf.app.flags.DEFINE_integer('depth', 3 , 'Number of layers in each encoder and decoder')
tf.app.flags.DEFINE_integer('embedding_size', 200, 'Embedding dimensions of encoder and decoder inputs')
tf.app.flags.DEFINE_integer('num_encoder_symbols', len(source_int_to_vocab), 'Source vocabulary size')
tf.app.flags.DEFINE_integer('num_decoder_symbols', len(target_int_to_vocab), 'Target vocabulary size')
tf.app.flags.DEFINE_string('ds_csv_path', 'cleaned_pw_paths_tr_10000.csv', 'path to the csv file containing the dataset')
tf.app.flags.DEFINE_boolean('use_residual', True, 'Use residual connection between layers')
# tf.app.flags.DEFINE_boolean('attn_input_feeding', False, 'Use input feeding method in attentional decoder')
tf.app.flags.DEFINE_boolean('use_dropout', True, 'Use dropout in each rnn cell')
tf.app.flags.DEFINE_float('dropout_rate', 0.4, 'Dropout probability for input/output/state units (0.0: no dropout)')
tf.app.flags.DEFINE_integer('edit_distance', 3, 'Filter out samples with edit distance greater than this number')
# Training parameters
tf.app.flags.DEFINE_float('learning_rate', 0.001, 'Learning rate')
tf.app.flags.DEFINE_float('max_gradient_norm', 1.0, 'Clip gradients to this norm')
tf.app.flags.DEFINE_integer('batch_size', 50, 'Batch size')
tf.app.flags.DEFINE_integer('max_epochs', 500, 'Maximum # of training epochs')
# tf.app.flags.DEFINE_integer('max_load_batches', 20, 'Maximum # of batches to load at one time')
tf.app.flags.DEFINE_integer('max_seq_length', 62, 'Maximum sequence length')
tf.app.flags.DEFINE_integer('display_freq', 10, 'Display training status every this iteration')
tf.app.flags.DEFINE_integer('save_freq', 11500, 'Save model checkpoint every this iteration')
tf.app.flags.DEFINE_integer('valid_freq', 40, 'Evaluate model every this iteration: valid_data needed')
tf.app.flags.DEFINE_string('optimizer', 'adam', 'Optimizer for training: (adadelta, adam, rmsprop)')
tf.app.flags.DEFINE_string('model_dir', 'model/', 'Path to save model checkpoints')
tf.app.flags.DEFINE_string('summary_dir', 'model/summary', 'Path to save model summary')
tf.app.flags.DEFINE_string('model_name', 'pass2path.ckpt', 'File name used for model checkpoints')
# tf.app.flags.DEFINE_boolean('shuffle_each_epoch', True, 'Shuffle training dataset for each epoch')
# tf.app.flags.DEFINE_boolean('sort_by_length', True, 'Sort pre-fetched minibatches by their target sequence lengths')
tf.app.flags.DEFINE_boolean('use_fp16', False, 'Use half precision float16 instead of float32 as dtype')

# Runtime parameters
tf.app.flags.DEFINE_boolean('allow_soft_placement', True, 'Allow device soft placement')
tf.app.flags.DEFINE_boolean('log_device_placement', False, 'Log placement of ops on devices')

FLAGS = tf.app.flags.FLAGS

In [6]:
def set_train_flags(cell_type='lstm', hidden_units=128, num_layers=3, embed_size=200, path_to_ds='cleaned_pw_paths_tr.csv',
                   use_residual=False, keep_prob=0.6, edit_distance=2, lr=0.001, batch_size=50, epochs=10,
                   display_step=100, save_freq=11500):
    
    tf.app.flags.FLAGS.__flags.clear()

    # Network parameters
    tf.app.flags.DEFINE_string('cell_type', cell_type , 'RNN cell for encoder and decoder, default: lstm')
    tf.app.flags.DEFINE_integer('hidden_units', hidden_units, 'Number of hidden units in each layer')
    tf.app.flags.DEFINE_integer('depth', num_layers , 'Number of layers in each encoder and decoder')
    tf.app.flags.DEFINE_integer('embedding_size', embed_size, 'Embedding dimensions of encoder and decoder inputs')
    tf.app.flags.DEFINE_integer('num_encoder_symbols', len(source_int_to_vocab), 'Source vocabulary size')
    tf.app.flags.DEFINE_integer('num_decoder_symbols', len(target_int_to_vocab), 'Target vocabulary size')
    tf.app.flags.DEFINE_string('ds_csv_path', path_to_ds, 'path to the csv file containing the dataset')
    tf.app.flags.DEFINE_boolean('use_residual', use_residual, 'Use residual connection between layers')
    tf.app.flags.DEFINE_boolean('use_dropout', True, 'Use dropout in each rnn cell')
    tf.app.flags.DEFINE_float('dropout_rate', 1 - keep_prob, 'Dropout probability for input/output/state units (0.0: no dropout)')
    tf.app.flags.DEFINE_integer('edit_distance', edit_distance, 'Filter out samples with edit distance greater than this number')
    # Training parameters
    tf.app.flags.DEFINE_float('learning_rate', lr, 'Learning rate')
    tf.app.flags.DEFINE_float('max_gradient_norm', 1.0, 'Clip gradients to this norm')
    tf.app.flags.DEFINE_integer('batch_size', batch_size, 'Batch size')
    tf.app.flags.DEFINE_integer('max_epochs', epochs, 'Maximum # of training epochs')
    # tf.app.flags.DEFINE_integer('max_load_batches', 20, 'Maximum # of batches
    # to load at one time')
    tf.app.flags.DEFINE_integer('max_seq_length', 62, 'Maximum sequence length')
    tf.app.flags.DEFINE_integer('display_freq', display_step, 'Display training status every this iteration')
    tf.app.flags.DEFINE_integer('save_freq', save_freq, 'Save model checkpoint every this iteration')
    tf.app.flags.DEFINE_integer('valid_freq', 3 * display_step, 'Evaluate model every this iteration: valid_data needed')
    tf.app.flags.DEFINE_string('optimizer', 'adam', 'Optimizer for training: (adadelta, adam, rmsprop)')
    tf.app.flags.DEFINE_string('model_dir', './model/', 'Path to save model checkpoints')
    tf.app.flags.DEFINE_string('summary_dir', './model/summary', 'Path to save model summary')
    tf.app.flags.DEFINE_string('model_name', 'pass2path.ckpt', 'File name used for model checkpoints')
    # Ignore Cmmand Line
    tf.app.flags.DEFINE_string('t', '', '')
    tf.app.flags.DEFINE_string('d', '', '')
    tf.app.flags.DEFINE_string('b', '', '')
    tf.app.flags.DEFINE_string('q', '', '')
    tf.app.flags.DEFINE_string('c', '', '')
    tf.app.flags.DEFINE_string('s', '', '')
    tf.app.flags.DEFINE_string('e', '', '')
    tf.app.flags.DEFINE_string('z', '', '')
    tf.app.flags.DEFINE_string('l', '', '')
    tf.app.flags.DEFINE_string('m', '', '')
    tf.app.flags.DEFINE_string('w', '', '')
    tf.app.flags.DEFINE_string('i', '', '')
    tf.app.flags.DEFINE_string('f', '', '')
    tf.app.flags.DEFINE_string('r', '', '')
    tf.app.flags.DEFINE_string('k', '', '')
    # tf.app.flags.DEFINE_boolean('shuffle_each_epoch', True, 'Shuffle training
    # dataset for each epoch')
    # tf.app.flags.DEFINE_boolean('sort_by_length', True, 'Sort pre-fetched
    # minibatches by their target sequence lengths')
    tf.app.flags.DEFINE_boolean('use_fp16', False, 'Use half precision float16 instead of float32 as dtype')

    # Runtime parameters
    tf.app.flags.DEFINE_boolean('allow_soft_placement', True, 'Allow device soft placement')
    tf.app.flags.DEFINE_boolean('log_device_placement', False, 'Log placement of ops on devices')

In [7]:
# Globals
kb = Keyboard()
# thisfolder = Path(__file__).absolute().parent
# TRANS_to_IDX = json.load((thisfolder / 'data/trans_dict_2idx.json').open()) # py
with open(('./data/trans_dict_2idx.json')) as f:
    TRANS_to_IDX = json.load(f) # jupyter
IDX_to_TRANS = {v: literal_eval(k) for k, v in TRANS_to_IDX.items()}
char_lst = list(string.ascii_letters) + list(string.digits) + list(string.punctuation) + [" ", "\t", "\x03", "\x04"]
source_vocab_to_int, source_int_to_vocab = create_lookup_tables_from_lst(char_lst)
target_vocab_to_int, target_int_to_vocab = build_path_vocab(TRANS_to_IDX)
NUM_SAMPLES_LARGE_DS = 117574260
set_train_flags(path_to_ds='cleaned_pw_paths_tr_10000.csv')
FLAGS = tf.app.flags.FLAGS

In [8]:
def create_model(session, FLAGS):

    config = OrderedDict(sorted((dict([(key,val.value) for key,val in FLAGS.__flags.items()])).items()))
    model = Pass2PathModel(config, 'train')

    ckpt = tf.train.get_checkpoint_state(FLAGS.model_dir)
    if (ckpt):
        print("Found a checkpoint state...")
        print(ckpt.model_checkpoint_path)
    if (ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path)):
        print('Reloading model parameters..')
        model.restore(session, ckpt.model_checkpoint_path)
        
    else:
        if not os.path.exists(FLAGS.model_dir):
            os.makedirs(FLAGS.model_dir)
        print('Created new model parameters..')
        session.run(tf.global_variables_initializer())
   
    return model

In [5]:
def input_to_ids(source_pws_lst, target_paths_lst, source_vocab_to_int, target_vocab_to_int):
    """
    1st, 2nd args: lists passwords and paths to be converted
    3rd, 4th args: lookup tables for 1st and 2nd args respectively   
    return: A tuple of lists (source_id_pass, target_id_path) converted
    Will use for mini-batches of (passwords, paths)
    """
    # empty list of converted passwords and paths
    source_pass_id = []
    target_path_id = []
    
    max_source_pass_length = max([len(password) for password in source_pws_lst])
    max_target_path_length = max([len(path) for path in target_paths_lst])
    
    # iterating through each password (# of passwords & paths is the same)
    for i in range(len(source_pws_lst)):
        # extract password, one by one
        source_password = source_pws_lst[i]
#         print(target_paths_lst)
        target_path = target_paths_lst[i]
        
        # make a list of tokens/words (extraction) from the chosen password
        source_tokens = list(source_password)
        target_tokens = [str(t) for t in target_path]
        
        # empty list of converted words to index in the chosen password
        source_token_id = []
        target_token_id = []
        
        for index, token in enumerate(source_tokens):
            if (token != ""):
                if (source_vocab_to_int.get(token)):
                    source_token_id.append(source_vocab_to_int[token])
        
        for index, token in enumerate(target_tokens):
            if (token != ""):
                if (target_vocab_to_int.get(token)):
                    target_token_id.append(target_vocab_to_int[token])
                
        # put <EOS> token at the end of the chosen target sentence
        # this token suggests when to stop creating a sequence
        target_token_id.append(target_vocab_to_int['<EOS>'])
            
        # add each converted sentences in the final list
        source_pass_id.append(source_token_id)
        target_path_id.append(target_token_id)
    
    return source_pass_id, target_path_id

def _parse_line(line):
    '''
    This helper function parses a line from the CSV dataset to a dictionary and a target.
    '''
    # Metadata describing the text columns
    
    COLUMNS = ['pass_1', 'pass_2', 'path']
    FIELD_DEFAULTS = [[""], [""], [""]]
    
    # Decode the line into its fields
    fields = tf.decode_csv(line, FIELD_DEFAULTS)

    # Pack the result into a dictionary
    features = dict(zip(COLUMNS,fields))

    # Separate the label from the features
    path = features.pop('path')

    return features, path

def csv_input_fn(csv_path, batch_size, skip_lines):
    '''
    This function builds a batch from the dataset in the CSV file.
    '''
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path).skip(skip_lines)

    # Parse each line.
    dataset = dataset.map(_parse_line)

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

def csv_input_fn_filter(csv_path, batch_size, skip_lines, edit_distance, limit=-1):
    '''
    This function builds a batch from the dataset in the CSV file, and filters out
    samples that do not satisfy the edit distance condition.
    '''
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path).skip(skip_lines).take(limit)

    # Parse each line.
    dataset = dataset.map(_parse_line)
    
    # Filter
    dataset = dataset.filter(lambda feat, path : tf.less_equal(tf.size((tf.string_split(tf.reshape(path, shape=(1,)), " []"))), edit_distance))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

def pad_sequence_batch(sequence_batch, pad_int):
    """
    Pad sequences with <PAD> = <EOS> so that each sequence of a batch has the same length
    """
    max_sequence = max([len(seq) for seq in sequence_batch])
    return [seq + [pad_int] * (max_sequence - len(seq)) for seq in sequence_batch]

def preprocess_batch(batch, source_vocab_to_int, target_vocab_to_int):
    #kb = Keyboard()
    pass_batch = [kb.word_to_keyseq(d.decode('utf-8')) for d in batch[0]['pass_1']]
    pass2_batch = [kb.word_to_keyseq(d.decode('utf-8')) for d in batch[0]['pass_2']]
    path_batch = [json.loads(j) for j in [d.decode('utf-8') for d in batch[1]]]
    pass_batch_ids, path_batch_ids = input_to_ids(pass_batch, path_batch,
                                                      source_vocab_to_int, target_vocab_to_int)
    source_pad_int = source_vocab_to_int['<EOS>']
    target_pad_int = target_vocab_to_int['<EOS>']
    # Pad
    pad_sources_batch = np.array(pad_sequence_batch(pass_batch_ids, source_pad_int))
    pad_targets_batch = np.array(pad_sequence_batch(path_batch_ids, target_pad_int))

    # Need the lengths for the _lengths parameters
    pad_targets_lengths = []
    for target in pad_targets_batch:
        pad_targets_lengths.append(len(target))

    pad_source_lengths = []
    for source in pad_sources_batch:
        pad_source_lengths.append(len(source))
    return (np.array(pad_sources_batch), np.array(pad_targets_batch), 
            np.array(pad_source_lengths), np.array(pad_targets_lengths), pass_batch, pass2_batch)

def get_accuracy_beam(target, logits, pad_int=0):
    """
    Calculate accuracy of BeamSearch output as follows: if one of the K outputs is correct,
    it counts as a correct prediction (positive contribution to the accuracy).
    "logits" are now of the shape: [batches, max_seq, K]
    """
#     print(target.shape)
#     print(logits.shape)
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant', constant_values = pad_int)
    if max_seq - logits.shape[1]:
        logits = np.pad(logits,
            [(0,0), (0,max_seq - logits.shape[1]), (0,0)],
            'constant', constant_values = pad_int)
#     print(target)
#     print(logits)
    acc = [np.mean(np.equal(target, np.reshape(logits[:,:,i], target.shape))) for i in range(logits.shape[2])]
#     if (max(acc) > 0.6):
#         print(target)
#         print(np.reshape(np.transpose(logits[:,:,acc.index(max(acc))]),
#         target.shape))
    return max(acc)

def pass_to_seq(password, vocab_to_int):
    '''
    This function transforms password to sequence of integers, in order to make the tested password
    applicable to the network's input.
    '''
    results = []
    for c in list(password):
        if c in vocab_to_int:
            results.append(vocab_to_int[c])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results

def path2word_kb_feasible(word, path, print_path=False):
    '''
    This function decodes the word in which the given path transitions the input word into.
    This is the KeyPress version, which handles the keyboard representations.
    If one of the parts components is not feasible (e.g removing a char from out of range index), it skips it
    Input parameters: original word, transition path
    Output: decoded word
    '''
    #kb = Keyboard()
    word = kb.word_to_keyseq(word)
    if not path:
        return kb.keyseq_to_word(word)
    #path = [literal_eval(p) for p in path]
    if (print_path):
        print(path)
#     print(type(path))
#     print(word)
    final_word = []
    word_len = len(word)
    path_len = len(path)
    i = 0
    j = 0
    while (i < word_len or j < path_len):
        if ((j < path_len and path[j][2] == i) or (i >= word_len and path[j][2] >= i)):
            if (path[j][0] == "s"):
                # substitute
                final_word.append(path[j][1])
                i += 1
                j += 1
            elif (path[j][0] == "d"):
                # delete
                i += 1
                j += 1
            else:
                # "i", insert
                final_word.append(path[j][1])
                j += 1
        else:
            if (i < word_len):
                final_word.append(word[i])
                i += 1
            if (j < path_len and i > path[j][2]):
                j += 1
    return (kb.keyseq_to_word(''.join(final_word)))

def path_to_pass(password, idx_path, trans_dict):
    '''
    This function decodes the password in which the given path transitions the input password into.
    Input parameters: original password, transition path, transition dictionary
    Output: decoded password
    '''
#     print(idx_path)
    str_path = []
    for i in idx_path:
        if (i != '<PAD>' and i != '<UNK>') and trans_dict.get(int(i)):
            str_path.append(trans_dict[int(i)])
#         else:
#             print("could not find " + str(i) + " in dictionary")
#     print(str_path)
    output_pass = path2word_kb_feasible(password, str_path)
    return output_pass

def get_accuracy_beam_decode(logits, pass1_batch, pass2_batch, target_int_to_vocab, trans_dict_2path, bias=False):
    """
    Calculate accuracy of BeamSearch output as follows: if one of the K outputs is correct,
    it counts as a correct prediction (positive contribution to the accuracy).
    "logits" are now of the shape: [batches, max_seq, K]
    """
#     print(logits.shape)
    beam_width = logits.shape[2]
    match_vec = np.zeros((1, logits.shape[0]), dtype=bool)
    for i in range(beam_width):
        decode_pred_batch = logits[:,:,i]
        for k in range(logits.shape[0]):
            decode_pred = logits[k,:,i]
#             print(decode_pred)
            path_idx_pred = [target_int_to_vocab[j] for j in decode_pred]
#             print(path_idx_pred)
            if ('<EOS>' in path_idx_pred):
                path_idx_pred = path_idx_pred[:path_idx_pred.index('<EOS>')]
            prediction = path_to_pass(pass1_batch[k], path_idx_pred, trans_dict_2path)
#             print("pred: {}, targ: {}".format(prediction, pass2_batch[k]))
            if (prediction == kb.keyseq_to_word(pass2_batch[k])):
#                 print(prediction)
                match_vec[0, k] = True
            if (bias and (not match_vec[0, k]) and (k == logits.shape[0] - 1)):
                if (kb.keyseq_to_word(pass2_batch[k]) == kb.keyseq_to_word(pass1_batch[k])):
                    match_vec[0, k] = True       
    acc = np.mean(match_vec)
#     if (max(acc) > 0.6):
#         print(target)
#         print(np.reshape(np.transpose(logits[:,:,acc.index(max(acc))]),
#         target.shape))
    return acc


In [21]:
def train():
    trans_dict_2path = IDX_to_TRANS
    #print(FLAGS.__flags.items())
    f_path = FLAGS.ds_csv_path
    batch_size = FLAGS.batch_size
    edit_distance = FLAGS.edit_distance
    # Initiate TF session
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement, 
        log_device_placement=FLAGS.log_device_placement, gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
        print('Loading training data..')
        if ("cleaned_pw_paths_tr.csv" in f_path and edit_distance == 3):
            num_samples = NUM_SAMPLES_LARGE_DS
        elif ("cleaned_pw_paths_tr.l8c3.csv" in f_path and edit_distance == 3):
            num_samples = NUM_SAMPLES_CORNELL_DS
        else:
            num_samples = 0
            with open(f_path) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                num_samples = sum(1 for row in csv_reader if len(json.loads(row[2])) <= edit_distance)

        total_batches = num_samples // batch_size

        print("# Samples: {}".format(num_samples))
        print("Total batches: {}".format(total_batches))

        # Split data to training and validation sets
        num_validation = int(0.2 * num_samples)
        total_valid_batches = num_validation // batch_size
        total_train_batches = total_batches - total_valid_batches

        print("Total validation batches: {}".format(total_valid_batches))
        print("Total training batches: {}".format(total_train_batches))

        valid_dataset = csv_input_fn_filter(f_path, batch_size, skip_lines = 0,
                                            edit_distance = edit_distance, limit = num_validation)
        train_dataset = csv_input_fn_filter(f_path, batch_size, skip_lines = num_validation,
                                            edit_distance = edit_distance, limit = -1)

        valid_iterator = valid_dataset.make_initializable_iterator()
        train_iterator = train_dataset.make_initializable_iterator()

        next_valid_element = valid_iterator.get_next()
        next_train_element = train_iterator.get_next()

        sess.run(valid_iterator.initializer)
        sess.run(train_iterator.initializer)

        # Create a log writer object
        log_writer = tf.summary.FileWriter(FLAGS.model_dir, graph=sess.graph)
        
        # Create a new model or reload existing checkpoint
        model = create_model(sess, FLAGS)

        step_time, loss = 0.0, 0.0
        words_seen, sents_seen = 0, 0
        start_time = time.time()
        global_start_time = start_time

        # Training loop
        print('Training..')
        for epoch_idx in range(FLAGS.max_epochs):
            if (model.global_epoch_step.eval() >= FLAGS.max_epochs):
                print('Training is already complete.', \
                      'current epoch:{}, max epoch:{}'.format(model.global_epoch_step.eval(), FLAGS.max_epochs))
                break

            for batch_i in range(total_train_batches):    
                # Get a batch
                batch_train = sess.run(next_train_element)
                
        
                (source_batch, target_batch, sources_lengths, targets_lengths, pass1_train_batch,
                 pass2_train_batch) = preprocess_batch(batch_train, 
                                                       source_vocab_to_int,
                                                       target_vocab_to_int)

                # Execute a single training step
                step_loss, summary = model.train(sess, encoder_inputs=source_batch, encoder_inputs_length=sources_lengths, 
                                                 decoder_inputs=target_batch, decoder_inputs_length=targets_lengths)
#                 print(step_loss)

                loss += float(step_loss) / FLAGS.display_freq
                words_seen += float(np.sum(sources_lengths + targets_lengths))
                sents_seen += float(source_batch.shape[0]) # batch_size

                if (model.global_step.eval() % FLAGS.display_freq == 0):
                    # CHANGE TO ACCURACY
                    avg_perplexity = math.exp(float(loss)) if loss < 300 else float("inf")

                    time_elapsed = time.time() - start_time
                    step_time = time_elapsed / FLAGS.display_freq

                    words_per_sec = words_seen / time_elapsed
                    sents_per_sec = sents_seen / time_elapsed

                    print('Epoch ', model.global_epoch_step.eval(), 'Batch: {}/{}'.format(batch_i, total_train_batches) , 'Step ', model.global_step.eval(), \
                          'Step-time {0:.2f}'.format(step_time), 'Total-time: {0:.2f}'.format(time.time() - global_start_time), \
                          '{0:.2f} passwords/s'.format(sents_per_sec), '{0:.2f} tokens/s'.format(words_per_sec), \
                            'Loss: {0:.3f}'.format(loss))

                    loss = 0
                    words_seen = 0
                    sents_seen = 0
                    start_time = time.time()
                    
                    # Record training summary for the current batch
                    log_writer.add_summary(summary, model.global_step.eval())

                # Execute a validation step
                if (model.global_step.eval() % FLAGS.valid_freq == 0):
                    print('Validation step')
                    batch_valid = sess.run(next_valid_element)
                    (valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths,
                        pass1_valid_batch, pass2_valid_batch) = preprocess_batch(batch_valid, 
                                                                                 source_vocab_to_int,
                                                                                 target_vocab_to_int)
                    valid_loss = model.eval(sess, encoder_inputs=valid_sources_batch, encoder_inputs_length=valid_sources_lengths,
                                               decoder_inputs=valid_targets_batch, decoder_inputs_length=valid_targets_lengths)
                    print('Valid Loss: {0:.2f}'.format(valid_loss[0]))

                # Save the model checkpoint
                if (model.global_step.eval() % FLAGS.save_freq == 0):
                    print('Saving the model..')
                    checkpoint_path = os.path.join(FLAGS.model_dir, FLAGS.model_name)
                    model.save(sess, checkpoint_path, global_step=model.global_step)
                    json.dump(model.config,
                              open('%s-%d.json' % (checkpoint_path, model.global_step.eval()), 'w'),
                              indent=2)
            # Increase the epoch index of the model
            model.global_epoch_step_op.eval()
            print('Epoch {0:} DONE'.format(model.global_epoch_step.eval()))
        
        print('Saving the last model..')
        checkpoint_path = os.path.join(FLAGS.model_dir, FLAGS.model_name)
        model.save(sess, checkpoint_path, global_step=model.global_step)
        json.dump(model.config,
                  open('%s-%d.json' % (checkpoint_path, model.global_step.eval()), 'w'),
                  indent=2)
        total_time = time.time() - global_start_time
        
    print('Training Terminated, Total time: {} seconds'.format(total_time))

In [20]:
def main(_):
    train()


if __name__ == '__main__':
    tf.reset_default_graph()
    tf.app.run()

Loading training data..
# Samples: 1156
Total batches: 23
Total validation batches: 4
Total training batches: 19
building model..
building encoder..
building decoder and attention..
setting optimizer..
Reloading model parameters..
INFO:tensorflow:Restoring parameters from model/pass2path.ckpt-5600
model restored from model/pass2path.ckpt-5600
Training..
Epoch  450 Step  5610 Perplexity 217.58 Step-time 0.19 257.36 sents/s 5481.70 tokens/s Loss: 5.383
Epoch 451 DONE
Epoch  451 Step  5620 Perplexity 12.45 Step-time 0.05 991.22 sents/s 17048.90 tokens/s Loss: 2.521
Epoch  451 Step  5630 Perplexity 6.34 Step-time 0.08 662.76 sents/s 11730.94 tokens/s Loss: 1.847
Epoch 452 DONE
Epoch  452 Step  5640 Perplexity 5.40 Step-time 0.05 989.27 sents/s 18895.03 tokens/s Loss: 1.687
Validation step
Valid Loss: 3.076057195663452
Epoch  452 Step  5650 Perplexity 4.31 Step-time 0.10 494.39 sents/s 9344.04 tokens/s Loss: 1.460
Epoch 453 DONE
Epoch  453 Step  5660 Perplexity 3.49 Step-time 0.05 994.15 se

Epoch  485 Step  6280 Perplexity 1.48 Step-time 0.05 1041.39 sents/s 17911.96 tokens/s Loss: 0.393
Validation step
Valid Loss: 3.7619235515594482
Epoch 486 DONE
Epoch  486 Step  6290 Perplexity 1.46 Step-time 0.08 616.58 sents/s 11961.71 tokens/s Loss: 0.381
Epoch  486 Step  6300 Perplexity 1.46 Step-time 0.05 950.15 sents/s 19953.23 tokens/s Loss: 0.380
Epoch 487 DONE
Epoch  487 Step  6310 Perplexity 1.45 Step-time 0.07 680.67 sents/s 11775.52 tokens/s Loss: 0.371
Epoch  487 Step  6320 Perplexity 1.44 Step-time 0.05 990.32 sents/s 18717.06 tokens/s Loss: 0.365
Validation step
Valid Loss: 3.500795602798462
Epoch 488 DONE
Epoch  488 Step  6330 Perplexity 1.47 Step-time 0.08 633.57 sents/s 12418.02 tokens/s Loss: 0.385
Epoch  488 Step  6340 Perplexity 1.44 Step-time 0.05 938.64 sents/s 19711.50 tokens/s Loss: 0.366
Epoch 489 DONE
Epoch  489 Step  6350 Perplexity 1.46 Step-time 0.07 682.55 sents/s 11398.57 tokens/s Loss: 0.381
Epoch  489 Step  6360 Perplexity 1.41 Step-time 0.05 1028.64 s

SystemExit: 

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
FLAGS.__flags.items()

dict_items([('num_encoder_symbols', <absl.flags._flag.Flag object at 0x0000026459CD4C50>), ('hidden_units', <absl.flags._flag.Flag object at 0x0000026459CD4E48>), ('log_device_placement', <absl.flags._flag.BooleanFlag object at 0x0000026459AA5198>), ('batch_size', <absl.flags._flag.Flag object at 0x0000026459CD46A0>), ('num_decoder_symbols', <absl.flags._flag.Flag object at 0x0000026459CD4BA8>), ('use_fp16', <absl.flags._flag.BooleanFlag object at 0x0000026459CD40F0>), ('edit_distance', <absl.flags._flag.Flag object at 0x0000026459CD4898>), ('save_freq', <absl.flags._flag.Flag object at 0x0000026459CD4400>), ('model_dir', <absl.flags._flag.Flag object at 0x0000026459CD4208>), ('dropout_rate', <absl.flags._flag.Flag object at 0x0000026459CD4940>), ('display_freq', <absl.flags._flag.Flag object at 0x0000026459CD44A8>), ('use_residual', <absl.flags._flag.BooleanFlag object at 0x0000026459CD4AC8>), ('allow_soft_placement', <absl.flags._flag.BooleanFlag object at 0x0000026459CD4080>), ('use